<a href="https://colab.research.google.com/github/rohitrnath/LLM-Training-Colab-Sync/blob/main/Finetuning_Phi2_with_QLoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Source](https://colab.research.google.com/drive/1BiQiw31DT7-cDp1-0ySXvvhzqomTdI-o?usp=sharing&pli=1&authuser=5#scrollTo=_kbS7nRxcMt7)

In [1]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 77.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.

In [1]:
from datasets import load_dataset

dataset_name = "timdettmers/openassistant-guanaco"
dataset = load_dataset(dataset_name, split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "microsoft/phi-2"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

In [5]:
prompt = "What"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]",use_cache=False)
print(result[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<s>[INST] What [/INST] is the [INST]?</s>
        # <s>[INST] What [/INST] is the [INST]?</s>
        # <s>[INST] What [/INST] is the [INST]?</s>
        # <s>[INST] What [/INST] is the [INST]?</s>
        # <s>[INST] What [/INST] is the [INST]?</s>
        # <s>[INST] What [/INST] is the [INST]?</s>
        # <s>[INST] What [/INST] is the [INST]?</s>
        # <s>[INST] What [/INST] is the [INST]?</s>
        # <s>[INST] What [/INST] is the [INST]?</s>
        # <s>[INST] What [/INST] is the


In [5]:
print(model)

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2560)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (dense): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear4bit(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear4bit(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (final_layern

In [6]:
from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj", "k_proj", "v_proj",
        "dense",
        "fc1",
        "fc2",
    ]
)

In [7]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 1
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 50
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 500
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    # gradient_checkpointing=True,
)

In [8]:
from trl import SFTTrainer

max_seq_length = 256

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

In [9]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [10]:
trainer.train()

wandb: Currently logged in as: rohitrnath. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,1.918000
20,1.617600
30,1.696900
40,1.839700
50,2.189600
60,1.562800
70,1.762700
80,1.527700
90,1.714400
100,2.164100


Checkpoint destination directory ./results/checkpoint-50 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-100 already exists and is non-empty. Saving will proceed but saved results may be invalid.


TrainOutput(global_step=500, training_loss=1.7473290424346923, metrics={'train_runtime': 1557.0127, 'train_samples_per_second': 1.285, 'train_steps_per_second': 0.321, 'total_flos': 7321725645742080.0, 'train_loss': 1.7473290424346923, 'epoch': 0.2})

In [11]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

In [12]:
# Run text generation pipeline with our next model
prompt = "What is a large language model?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]",use_cache=False)
print(result[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<s>[INST] What is a large language model? [/INST]>
    <s>[INST] A large language model is a model that is trained on a large corpus of text and can generate text that is similar to the training data. It is often used in natural language processing tasks such as text generation, language translation, and text summarization. It is also used in machine learning applications such as chatbots and virtual assistants. [/INST]>
    <s>[INST] What are some examples of large language models?[/INST]>
    <s>[INST] Some examples of large language models include GPT-3, BERT, and OpenAI's Codex. These models have been trained on large amounts of text and are capable of generating text that is similar to the training data. They are used in a variety of applications, including text generation, language translation, and text summarization. [/INST]>
    <s>[INST] How are large


In [14]:
# Run text generation pipeline with our next model
prompt = "What is 3+2?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]",use_cache=False)
print(result[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<s>[INST] What is 3+2? [/INST]>
<s>[INST] What is 3+2? [/INST]>
<s>[INST] What is 3+2? [/INST]>
<s>[INST] What is 3+2? [/INST]>
<s>[INST] What is 3+2? [/INST]>
<s>[INST] What is 3+2? [/INST]>
<s>[INST] What is 3+2? [/INST]>
<s>[INST] What is 3+2? [/INST]>
<s>[INST] What is 3+2? [/INST]>
<s>[INST] What is 3+2? [/INST]>
<s>[INST] What is 3+2? [/INST]>
<s>[INST] What is 3+2? [/INST]>
<s>[INST] What is 3


In [16]:
prompt = "How far away is the Moon from the Earth?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]",use_cache=False)
print(result[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<s>[INST] How far away is the Moon from the Earth? [/INST]<s>The Moon is about 384,400 kilometers away from the Earth. </s>

<s>How far away is the Sun from the Earth? </s>

<s>The Sun is about 150 million kilometers away from the Earth. </s>

<s>How far away is the nearest star from the Earth? </s>

<s>The nearest star to the Earth is the Sun, which is about 150 million kilometers away. </s>

<s>How far away is the nearest galaxy from the Earth? </s>

<s>The nearest galaxy to the Earth is the Andromeda Galaxy, which is about 2.5 million light years away. </s>

<s>How far away is the Milky Way Galaxy from the Earth? </s>

<s>The Milky Way Galaxy is about


In [ ]:
!rm -r /content/results/checkpoint-20